In [ ]:
! pip install transformers
! pip install -U datasets
! pip install torch
! pip install torchdata

In [2]:
from transformers import AutoModelForSeq2SeqLM
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import GenerationConfig # Let's you define how model should generate text
import pprint # For nicer output formatting.
pp=pprint.PrettyPrinter(indent=4)


model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True) # use_fast=True -->load the "fast" version of the tokenizer if available from HuggingFace
dataset=load_dataset('gretelai/synthetic_text_to_sql')



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})

# Zero shot prompting

In [4]:
sample_index=[37,67,101,78]

for i, index in enumerate(sample_index):
  schema=dataset['test'][index]['sql_context']
  instruction=dataset['test'][index]['sql_prompt']
  sql=dataset['test'][index]['sql']


  prompt=f'''
        Translate the schema and description into SQL query.Respond only to the sql query without any additional characters.
        Schema:{schema}
        Instruction:{instruction}
        SQL Query:

  '''


  input=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(
      model.generate(
          input['input_ids'],
          max_new_tokens=50,
      )[0],
      skip_special_tokens=True
  )


  print('Example',i+1)
  print(f'Task \n{instruction}\n')
  print(f'Schema \n{schema}\n')
  print(f'Correct SQL query \n {sql}\n')
  print(f'Model GENERATION SQL- zero prompt \n {output}\n')

Example 1
Task 
What is the number of medical supplies distributed by each organization, in East Africa, for the last 3 years, and the total cost of the supplies?

Schema 
CREATE TABLE medical_supplies (supply_id INT, organization_id INT, location VARCHAR(255), supply_type VARCHAR(255), supply_cost DECIMAL(10,2), distribution_date DATE); INSERT INTO medical_supplies VALUES (1, 1, 'Country A', 'Medicine', 5000, '2020-01-01'); INSERT INTO medical_supplies VALUES (2, 1, 'Country A', 'Medical Equipment', 7000, '2021-01-01'); INSERT INTO medical_supplies VALUES (3, 2, 'Country B', 'Vaccines', 10000, '2021-01-01'); INSERT INTO medical_supplies VALUES (4, 2, 'Country B', 'First Aid Kits', 8000, '2020-01-01');

Correct SQL query 
 SELECT organization_id, location as region, COUNT(*) as number_of_supplies, SUM(supply_cost) as total_supply_cost FROM medical_supplies WHERE location = 'East Africa' AND distribution_date >= DATE_SUB(CURRENT_DATE, INTERVAL 3 YEAR) GROUP BY organization_id, location;

# One Shot Interface

In [5]:
# So we are giving our model a pattern with example, Telling do same operation and provide the output the pattern according to the eample

def create_one_shot_prmopt(schema:str,instruction:str,sample_index:list) ->str:
  prompt=f"""
      Translate schema and description into SQl query.Respond only to the SQL query without any additional characters.
      Prepare result for the last example based on previous examples below.
  """

  for indes in sample_index:
    example_schema=dataset['test'][index]['sql_context']
    example_instruction=dataset['test'][index]['sql_prompt']
    example_domain=dataset['test'][index]['domain']
    example_sql=dataset['test'][index]['sql']

    prompt+=f"""
      Schema:{example_schema}
      Domain:{example_domain}
      Instruction:{example_instruction}
      Query:{example_sql}
            """

    prompt+=f"""
      Schema:{schema}
      Instruction:{instruction}
      Query:
            """

  return prompt

In [6]:
example_index=[20]
index_to_proces=78

one_shot_prompt=create_one_shot_prmopt(
    dataset['test'][index_to_proces]['sql_context'],
    dataset['test'][index_to_proces]['sql_prompt'],
    example_index
)

print(one_shot_prompt)

sql=dataset['test'][index_to_proces]['sql']

inputs=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(
    model.generate(
        inputs['input_ids'],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(f'Correct SQL:{sql}')
print(f'Model Generation - zero prompt \n{output}\n')


      Translate schema and description into SQl query.Respond only to the SQL query without any additional characters.
      Prepare result for the last example based on previous examples below.
  
      Schema:CREATE TABLE unions (id INT, name TEXT); CREATE TABLE workers (id INT, union_id INT, industry TEXT, wage FLOAT); INSERT INTO unions (id, name) VALUES (1, 'Union Z'), (2, 'Union AA'), (3, 'Union AB'); INSERT INTO workers (id, union_id, industry, wage) VALUES (1, 1, 'retail', 500), (2, 1, 'retail', 550), (3, 2, 'retail', 600), (4, 2, 'retail', 650), (5, 3, 'retail', 700), (6, 3, 'retail', 750);
      Domain:trade unions
      Instruction:What is the maximum weekly wage for workers in the 'retail' industry in unions?
      Query:SELECT MAX(wage) FROM workers JOIN unions ON workers.union_id = unions.id WHERE industry = 'retail';
            
      Schema:CREATE TABLE unions (id INT, name TEXT); CREATE TABLE workers (id INT, union_id INT, industry TEXT, wage FLOAT); INSERT INTO unio

In [ ]:
example_indexes = list(range(60,70))
index_to_process = 78

prompt = create_one_shot_prmopt(
    dataset['test'][index_to_process]['sql_context'],
    dataset['test'][index_to_process]['sql_prompt'],
    example_indexes
)

print(one_shot_prompt)

sql = dataset['test'][index_to_process]['sql']

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(f'CORRECT SQL:\n{sql}')
print(f'MODEL GENERATION SQL - ZERO SHOT:\n{output}\n')

Token indices sequence length is longer than the specified maximum sequence length for this model (4739 > 512). Running this sequence through the model will result in indexing errors



      Translate schema and description into SQl query.Respond only to the SQL query without any additional characters.
      Prepare result for the last example based on previous examples below.
  
      Schema:CREATE TABLE unions (id INT, name TEXT); CREATE TABLE workers (id INT, union_id INT, industry TEXT, wage FLOAT); INSERT INTO unions (id, name) VALUES (1, 'Union Z'), (2, 'Union AA'), (3, 'Union AB'); INSERT INTO workers (id, union_id, industry, wage) VALUES (1, 1, 'retail', 500), (2, 1, 'retail', 550), (3, 2, 'retail', 600), (4, 2, 'retail', 650), (5, 3, 'retail', 700), (6, 3, 'retail', 750);
      Domain:trade unions
      Instruction:What is the maximum weekly wage for workers in the 'retail' industry in unions?
      Query:SELECT MAX(wage) FROM workers JOIN unions ON workers.union_id = unions.id WHERE industry = 'retail';
            
      Schema:CREATE TABLE unions (id INT, name TEXT); CREATE TABLE workers (id INT, union_id INT, industry TEXT, wage FLOAT); INSERT INTO unio